In [2]:
# Chargement package
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [3]:
# Connexion à ggdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Import du fichier via ggdrive
file_path = "/content/drive/MyDrive/Python M2/train_clean.csv" # Lien perso ggdrive
df = pd.read_csv(file_path, sep = ',',index_col = [0])

,iid_pid,gender,condtn,samerace,race_o,field_cd,race,goal,date,go_out,...,shopping,yoga,exphappy,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,match
0,6804,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0
1,6803,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0
2,6802,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0
3,6801,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0
4,6800,1,2,0,1.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0


## Arbre de décisions

In [9]:
df_clean = df.copy()

In [10]:
df_clean.head()

,iid_pid,gender,condtn,samerace,race_o,field_cd,race,goal,date,go_out,...,shopping,yoga,exphappy,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,match
0,6804,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0
1,6803,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0
2,6802,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0
3,6801,1,2,0,2.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0
4,6800,1,2,0,1.0,1.0,4.0,2.0,6.0,5.0,...,4.0,1.0,6.0,30.0,30.0,30.0,5.0,0.0,5.0,0


In [11]:
X = df_clean[df_clean.columns[3:]] # On enlève la colonne ID et la colonne CLASSE
X.pop('match')
y = df_clean.match # Colonne var à prédire

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6804 entries, 0 to 6803
Data columns (total 50 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   samerace  6804 non-null   int64  
 1   race_o    6804 non-null   float64
 2   field_cd  6804 non-null   float64
 3   race      6804 non-null   float64
 4   goal      6804 non-null   float64
 5   date      6804 non-null   float64
 6   go_out    6804 non-null   float64
 7   career_c  6804 non-null   float64
 8   int_corr  6804 non-null   float64
 9   age_o     6804 non-null   float64
 10  pf_o_att  6804 non-null   float64
 11  pf_o_sin  6804 non-null   float64
 12  pf_o_int  6804 non-null   float64
 13  pf_o_fun  6804 non-null   float64
 14  pf_o_amb  6804 non-null   float64
 15  pf_o_sha  6804 non-null   float64
 16  attr_o    6804 non-null   float64
 17  sinc_o    6804 non-null   float64
 18  intel_o   6804 non-null   float64
 19  fun_o     6804 non-null   float64
 20  amb_o     6804 non-null   floa

In [13]:
print(X.shape, y.shape)

(6804, 50) (6804,)


In [14]:
y.value_counts()

0    5658
1    1146
Name: match, dtype: int64

In [15]:
# Utilisation du model_selection de sklearn
import sklearn
from sklearn import model_selection
# Affichage de la version
print(sklearn.__version__)

1.0.2


In [16]:
X_app, X_test, y_app, y_test = model_selection.train_test_split(X,y, train_size=0.7, random_state=0, stratify=y)


In [17]:
print(X_app.shape, X_test.shape, y_app.shape, y_test.shape)

(4762, 50) (2042, 50) (4762,) (2042,)


In [18]:
# Vérification de l'égalité de distribution entre y_app et y_test
y_app.value_counts(normalize=True)

0    0.831583
1    0.168417
Name: match, dtype: float64

In [19]:
# Vérification de l'égalité de distribution entre y_app et y_test
y_test.value_counts(normalize=True)

0    0.831538
1    0.168462
Name: match, dtype: float64

In [20]:
# Load libraries
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


In [21]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(random_state=0)

# Train Decision Tree Classifer
clf = clf.fit(X_app,y_app)

#Predict the response for test dataset
y_pred = clf.predict(X_test)


In [22]:
# Model Accuracy, how often is the classifier correct?
print("F1 Score:",metrics.f1_score(y_test, y_pred, average = "macro"))
print(metrics.confusion_matrix(y_test,y_pred))


F1 Score: 0.5879506873868068
[[1443  255]
 [ 230  114]]


In [24]:
X_app, X_test, y_app, y_test = model_selection.train_test_split(X,y, train_size=0.7, random_state=0, stratify=y)

In [25]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

In [26]:
sm = SMOTE(random_state=42, sampling_strategy=0.5, k_neighbors=5)
X_res, y_res = sm.fit_resample(X_app, y_app)

In [27]:
# Create Decision Tree classifer object
clf2 = DecisionTreeClassifier(random_state=0,min_samples_split=15,min_samples_leaf=20)

# Train Decision Tree Classifer
clf2 = clf2.fit(X_res, y_res)

#Predict the response for test dataset
y_pred2 = clf2.predict(X_test)


In [28]:
# Model Accuracy, how often is the classifier correct?
print("F1 Score:",metrics.f1_score(y_test, y_pred2, average = "macro"))
print(metrics.confusion_matrix(y_test,y_pred2))

F1 Score: 0.5900724000321198
[[1518  180]
 [ 250   94]]


In [29]:
from sklearn.ensemble import GradientBoostingClassifier

In [30]:
clf3 = GradientBoostingClassifier(n_estimators=175, learning_rate=1,max_depth=1, random_state=0,min_samples_split=15, min_samples_leaf=20).fit(X_res, y_res)
y_pred3 = clf3.predict(X_test)
print("F1 Score :",metrics.f1_score(y_test, y_pred3, average = "macro"))
print(metrics.confusion_matrix(y_test,y_pred3))

F1 Score : 0.6257123090787018
[[1585  113]
 [ 246   98]]
